## Total Energy Burned
Active energy burned and basal energy burned

In [1]:
import pandas as pd

# Load file into dataframe
folder_path = 'c:/Users/aoife/Documents/Project/DataTables/'

df_aeb = pd.read_csv(folder_path + 'active-energy-burned.csv', usecols=['participantId', 'timestamp', 'startTime', 'endTime', 'value'], parse_dates=['timestamp', 'startTime', 'endTime'])
df_beb = pd.read_csv(folder_path + 'basal-energy-burned.csv', usecols=['participantId', 'timestamp', 'startTime', 'endTime', 'value'], parse_dates=['timestamp', 'startTime', 'endTime'])

print(df_aeb.head())
print('----------------\n')
print(df_beb.head())

# Create a list of dataframes

energy_dfs = [df_aeb, df_beb]


                          participantId                 timestamp  \
0  b2571643-4aec-492f-bc7f-6f23c7fe239a 2018-10-03 12:45:22+00:00   
1  b2571643-4aec-492f-bc7f-6f23c7fe239a 2018-10-03 12:45:22+00:00   
2  b2571643-4aec-492f-bc7f-6f23c7fe239a 2018-10-03 12:45:22+00:00   
3  b2571643-4aec-492f-bc7f-6f23c7fe239a 2018-10-03 12:45:22+00:00   
4  b2571643-4aec-492f-bc7f-6f23c7fe239a 2018-10-03 12:45:22+00:00   

                  startTime                   endTime  value  
0 2018-10-03 10:44:19+00:00 2018-10-03 10:44:40+00:00  1.822  
1 2018-10-03 10:44:40+00:00 2018-10-03 10:45:41+00:00  0.711  
2 2018-10-03 10:45:41+00:00 2018-10-03 10:46:53+00:00  2.267  
3 2018-10-03 10:46:53+00:00 2018-10-03 10:47:54+00:00  2.569  
4 2018-10-03 10:47:54+00:00 2018-10-03 10:48:25+00:00  0.564  
----------------

                          participantId                 timestamp  \
0  b2571643-4aec-492f-bc7f-6f23c7fe239a 2018-10-03 12:45:23+00:00   
1  b2571643-4aec-492f-bc7f-6f23c7fe239a 2018-10-03 

In [2]:
# Create a new column which contains the difference between the start and end time

for df in energy_dfs:
    if 'startTime' and 'endTime' in df.columns:
        df['duration'] = df['endTime'] - df['startTime']
        
        df['duration'] = df['duration'].dt.total_seconds()

        
        print(df.head())





                          participantId                 timestamp  \
0  b2571643-4aec-492f-bc7f-6f23c7fe239a 2018-10-03 12:45:22+00:00   
1  b2571643-4aec-492f-bc7f-6f23c7fe239a 2018-10-03 12:45:22+00:00   
2  b2571643-4aec-492f-bc7f-6f23c7fe239a 2018-10-03 12:45:22+00:00   
3  b2571643-4aec-492f-bc7f-6f23c7fe239a 2018-10-03 12:45:22+00:00   
4  b2571643-4aec-492f-bc7f-6f23c7fe239a 2018-10-03 12:45:22+00:00   

                  startTime                   endTime  value  duration  
0 2018-10-03 10:44:19+00:00 2018-10-03 10:44:40+00:00  1.822      21.0  
1 2018-10-03 10:44:40+00:00 2018-10-03 10:45:41+00:00  0.711      61.0  
2 2018-10-03 10:45:41+00:00 2018-10-03 10:46:53+00:00  2.267      72.0  
3 2018-10-03 10:46:53+00:00 2018-10-03 10:47:54+00:00  2.569      61.0  
4 2018-10-03 10:47:54+00:00 2018-10-03 10:48:25+00:00  0.564      31.0  
                          participantId                 timestamp  \
0  b2571643-4aec-492f-bc7f-6f23c7fe239a 2018-10-03 12:45:23+00:00   
1  b25716

In [3]:
# Adjust the startTime to contain just the date

for df in energy_dfs:
    if 'startTime' in df.columns:
        df['startTime'] = pd.to_datetime(df['startTime'], format='%Y-%m-%dT%H:%M:%SZ')

        df['startDate'] = df['startTime'].dt.date

        print(df.head())
        print('----------------\n')

'''df_aeb['startTime'] = pd.to_datetime(df_aeb['startTime'], format='%Y-%m-%dT%H:%M:%SZ')

df_aeb['startDate'] = df_aeb['startTime'].dt.date

print(df_aeb.head())'''

                          participantId                 timestamp  \
0  b2571643-4aec-492f-bc7f-6f23c7fe239a 2018-10-03 12:45:22+00:00   
1  b2571643-4aec-492f-bc7f-6f23c7fe239a 2018-10-03 12:45:22+00:00   
2  b2571643-4aec-492f-bc7f-6f23c7fe239a 2018-10-03 12:45:22+00:00   
3  b2571643-4aec-492f-bc7f-6f23c7fe239a 2018-10-03 12:45:22+00:00   
4  b2571643-4aec-492f-bc7f-6f23c7fe239a 2018-10-03 12:45:22+00:00   

                  startTime                   endTime  value  duration  \
0 2018-10-03 10:44:19+00:00 2018-10-03 10:44:40+00:00  1.822      21.0   
1 2018-10-03 10:44:40+00:00 2018-10-03 10:45:41+00:00  0.711      61.0   
2 2018-10-03 10:45:41+00:00 2018-10-03 10:46:53+00:00  2.267      72.0   
3 2018-10-03 10:46:53+00:00 2018-10-03 10:47:54+00:00  2.569      61.0   
4 2018-10-03 10:47:54+00:00 2018-10-03 10:48:25+00:00  0.564      31.0   

    startDate  
0  2018-10-03  
1  2018-10-03  
2  2018-10-03  
3  2018-10-03  
4  2018-10-03  
----------------

                          

"df_aeb['startTime'] = pd.to_datetime(df_aeb['startTime'], format='%Y-%m-%dT%H:%M:%SZ')\n\ndf_aeb['startDate'] = df_aeb['startTime'].dt.date\n\nprint(df_aeb.head())"

In [4]:
# Get the total duration per day for each participant

for df in energy_dfs:
    if 'duration' in df.columns:
        df['duration'] = df.groupby(['participantId', 'startDate'])['duration'].transform('sum')
        df['value'] = df.groupby(['participantId', 'startDate'])['value'].transform('sum')

        # Drop duplicates
        df.drop_duplicates(subset=['participantId', 'startDate'], inplace=True)

        print(df.head())
        #print(df['value'].head())
        print('----------------\n')

'''df_aeb['duration'] = df_aeb.groupby(['participantId', 'startDate'])['duration'].transform('sum')
df_aeb['value'] = df_aeb.groupby(['participantId', 'startDate'])['value'].transform('sum')

print(df_aeb['duration'].head())'''

                            participantId                 timestamp  \
0    b2571643-4aec-492f-bc7f-6f23c7fe239a 2018-10-03 12:45:22+00:00   
137  be073dc4-f492-4cf3-a183-31cc15657a3f 2017-09-06 12:52:29+00:00   
180  be073dc4-f492-4cf3-a183-31cc15657a3f 2017-09-06 12:52:29+00:00   
435  be073dc4-f492-4cf3-a183-31cc15657a3f 2017-09-07 19:39:12+00:00   
590  be073dc4-f492-4cf3-a183-31cc15657a3f 2017-09-14 15:16:00+00:00   

                    startTime                   endTime    value  duration  \
0   2018-10-03 10:44:19+00:00 2018-10-03 10:44:40+00:00  112.882    8127.0   
137 2017-09-05 23:15:40+00:00 2017-09-05 23:16:42+00:00  130.400    4696.0   
180 2017-09-06 00:00:06+00:00 2017-09-06 00:00:26+00:00  195.420   12242.0   
435 2017-09-07 15:48:13+00:00 2017-09-07 15:49:14+00:00   78.150    8786.0   
590 2017-09-14 13:40:24+00:00 2017-09-14 13:41:27+00:00  128.663    5547.0   

      startDate  
0    2018-10-03  
137  2017-09-05  
180  2017-09-06  
435  2017-09-07  
590  2017-09-1

"df_aeb['duration'] = df_aeb.groupby(['participantId', 'startDate'])['duration'].transform('sum')\ndf_aeb['value'] = df_aeb.groupby(['participantId', 'startDate'])['value'].transform('sum')\n\nprint(df_aeb['duration'].head())"

In [5]:
# Calculate total energy burned and total duration per day

# Sort by participantId
df_aeb.sort_values(by=['participantId'], inplace=True)
df_beb.sort_values(by=['participantId'], inplace=True)

# Rename value columns
df_aeb.rename(columns={'value': 'active_energy'}, inplace=True)
df_beb.rename(columns={'value': 'basal_energy'}, inplace=True)

# Merge on participantId and startDate

df_energy = pd.merge(df_aeb, df_beb, on=['participantId', 'startDate'])

# Create a new column which contains the total energy burned per day

df_energy['total_energy'] = df_energy['active_energy'] + df_energy['basal_energy']

# Create a new column which contains the total duration per day

df_energy['total_duration'] = df_energy['duration_x'] + df_energy['duration_y']

# Rename duration columns
df_energy.rename(columns={'duration_x': 'active_duration'}, inplace=True)
df_energy.rename(columns={'duration_y': 'basal_duration'}, inplace=True)

# Drop unnecessary columns

df_energy.drop(columns=['timestamp_x', 'startTime_x', 'endTime_x', 'timestamp_y', 'startTime_y', 'endTime_y'], inplace=True)

print(df_energy.head())
print('----------------\n')
print(df_energy['total_duration'].head())



                          participantId  active_energy  active_duration  \
0  00fd4039-9b5e-4bbb-8295-4983a3f58371         13.815           5193.0   
1  017702d7-e540-4e1a-ab19-ca5f1c34e70b         41.148           7294.0   
2  01866570-f91f-4d3b-9d41-0a8cdcbfa922         32.569           8945.0   
3  0292a3a2-5d0b-42a7-ba86-03eae9cda7d5         13.806            901.0   
4  048ba877-5df6-47c4-b485-d3538db9e9b4         30.735           8045.0   

    startDate  basal_energy  basal_duration  total_energy  total_duration  
0  2018-08-15       227.847         14460.0       241.662         19653.0  
1  2017-12-23       879.016         45039.0       920.164         52333.0  
2  2018-02-25       685.827         30589.0       718.396         39534.0  
3  2017-11-24       933.963         37691.0       947.769         38592.0  
4  2017-11-17       593.295         29675.0       624.030         37720.0  
----------------

0    19653.0
1    52333.0
2    39534.0
3    38592.0
4    37720.0
Name: tota

## BMI

Body Mass and Height

In [6]:
# Use the groupby function to get the average height/body mass for each participant

df_bm = pd.read_csv(folder_path + 'body-mass.csv', usecols=['participantId', 'timestamp', 'value'], parse_dates=['timestamp'])
df_h = pd.read_csv(folder_path + 'height.csv', usecols=['participantId', 'timestamp', 'value'], parse_dates=['timestamp'])

bmi_dfs = [df_bm, df_h]

for df in bmi_dfs:
    if 'value' in df.columns:

        # Convert timestamp to datetime
        df['timestamp'] = pd.to_datetime(df['timestamp'], format='%Y-%m-%dT%H:%M:%SZ')
        # Convert timestamp to date

        # Get the average value for each participant
        df['value'] = df.groupby(['participantId'])['value'].transform('mean')

        # Drop duplicates
        df.drop_duplicates(subset=['participantId'], inplace=True)

        print(df.head())
        print('----------------\n')

                          participantId                 timestamp       value
0  11b2bf4d-6020-4a86-81bd-237c5616c649 2016-08-24 23:56:14+00:00   97.976000
1  9cbbb597-30c0-4f48-aebd-7fe58c627cf6 2016-03-16 02:33:20+00:00   63.503000
2  941e9fa4-6ba5-41f5-9bcc-805849d751b8 2017-01-01 14:07:00+00:00   96.666000
3  41c683b0-f6c4-4a6a-8dd2-4b9a3dd30a32 2016-03-30 02:24:59+00:00  112.846333
4  35fbae45-0f21-45ea-8ed7-8d3361a29b86 2016-07-18 17:45:38+00:00   68.690083
----------------

                          participantId                 timestamp   value
0  11b2bf4d-6020-4a86-81bd-237c5616c649 2016-08-24 23:56:14+00:00  1.6256
1  9cbbb597-30c0-4f48-aebd-7fe58c627cf6 2016-03-16 02:33:20+00:00  1.6510
2  941e9fa4-6ba5-41f5-9bcc-805849d751b8 2017-01-01 14:07:00+00:00  1.8288
3  1cba6631-3fb8-4c18-904f-ab56fe82833d 2016-03-07 05:54:00+00:00  1.5494
4  eb2c4ff0-54e2-4c34-b73d-003e3e0427ad 2016-04-02 07:17:47+00:00  1.7018
----------------



In [7]:
# Calculate the BMI for each participant and create a new df

# Sort by participantId
df_bm.sort_values(by=['participantId'], inplace=True)
df_h.sort_values(by=['participantId'], inplace=True)

# Rename value columns
df_bm.rename(columns={'value': 'bodyMass_kg'}, inplace=True)
df_h.rename(columns={'value': 'height_m'}, inplace=True)

# Merge on participantId

df_bmi = pd.merge(df_bm, df_h, on='participantId')

df_bmi['startDate'] = df_bmi['timestamp_x'].dt.date

df_bmi['bmi'] = df_bmi['bodyMass_kg'] / (df_bmi['height_m'] ** 2)

# drop unnecessary columns and duplicates
df_bmi.drop(columns=['timestamp_x', 'timestamp_y'], inplace=True)
df_bmi.drop_duplicates(subset=['participantId'], inplace=True)

print(df_bmi.head())


                          participantId  bodyMass_kg  height_m   startDate  \
0  00a55fb5-da33-4e2e-ae61-28f589fcc174       51.256    1.5748  2016-05-12   
1  00d6d2ee-ccea-45c7-9772-b19fd9bef2bf       74.389    1.9050  2018-04-20   
2  00fd4039-9b5e-4bbb-8295-4983a3f58371       59.874    1.6510  2018-08-15   
3  0113e483-0fc8-4892-a4fc-0b2f3820dde3       68.039    1.7018  2019-03-06   
4  013d82d7-b3cc-4007-b00c-4d1d75bab9dd       76.204    1.6002  2018-02-21   

         bmi  
0  20.667783  
1  20.498343  
2  21.965653  
3  23.493130  
4  29.759747  


## Heart Rate

In [8]:

df_hr = pd.read_csv(folder_path + 'heart-rate.csv', usecols=['participantId', 'timestamp', 'startTime', 'endTime', 'value'], parse_dates=['timestamp', 'startTime', 'endTime'])


In [9]:
# Convert startTime to datetime and extract date
df_hr['startTime'] = pd.to_datetime(df_hr['startTime'], format='%Y-%m-%dT%H:%M:%SZ')
df_hr['endTime'] = pd.to_datetime(df_hr['endTime'], format='%Y-%m-%dT%H:%M:%SZ')
df_hr['startDate'] = df_hr['startTime'].dt.date
print(df_hr.head()) 

                          participantId                 timestamp  \
0  06bc6ebb-a233-469f-8091-90256f656b1b 2016-03-13 19:25:18+00:00   
1  06bc6ebb-a233-469f-8091-90256f656b1b 2016-03-13 19:25:18+00:00   
2  06bc6ebb-a233-469f-8091-90256f656b1b 2016-03-13 19:25:18+00:00   
3  06bc6ebb-a233-469f-8091-90256f656b1b 2016-03-13 19:25:18+00:00   
4  06bc6ebb-a233-469f-8091-90256f656b1b 2016-03-13 19:25:18+00:00   

                  startTime                   endTime  value   startDate  
0 2016-03-13 13:52:58+00:00 2016-03-13 13:52:58+00:00  1.500  2016-03-13  
1 2016-03-13 13:53:00+00:00 2016-03-13 13:53:00+00:00  1.500  2016-03-13  
2 2016-03-13 13:53:01+00:00 2016-03-13 13:53:01+00:00  1.517  2016-03-13  
3 2016-03-13 13:53:02+00:00 2016-03-13 13:53:02+00:00  1.533  2016-03-13  
4 2016-03-13 13:53:03+00:00 2016-03-13 13:53:03+00:00  1.550  2016-03-13  


In [10]:


# Remove rows where participantId, startTime and endTime are duplicated

df_hr.drop_duplicates(subset=['participantId', 'startTime', 'endTime'], inplace=True)

print(df_hr.head())


                          participantId                 timestamp  \
0  06bc6ebb-a233-469f-8091-90256f656b1b 2016-03-13 19:25:18+00:00   
1  06bc6ebb-a233-469f-8091-90256f656b1b 2016-03-13 19:25:18+00:00   
2  06bc6ebb-a233-469f-8091-90256f656b1b 2016-03-13 19:25:18+00:00   
3  06bc6ebb-a233-469f-8091-90256f656b1b 2016-03-13 19:25:18+00:00   
4  06bc6ebb-a233-469f-8091-90256f656b1b 2016-03-13 19:25:18+00:00   

                  startTime                   endTime  value   startDate  
0 2016-03-13 13:52:58+00:00 2016-03-13 13:52:58+00:00  1.500  2016-03-13  
1 2016-03-13 13:53:00+00:00 2016-03-13 13:53:00+00:00  1.500  2016-03-13  
2 2016-03-13 13:53:01+00:00 2016-03-13 13:53:01+00:00  1.517  2016-03-13  
3 2016-03-13 13:53:02+00:00 2016-03-13 13:53:02+00:00  1.533  2016-03-13  
4 2016-03-13 13:53:03+00:00 2016-03-13 13:53:03+00:00  1.550  2016-03-13  


In [11]:
# Count the number of rows per person per day

df_hr['hr_duration_(s)'] = df_hr.groupby(['participantId', 'startDate'])['value'].transform('count')
df_hr['hr_duration_(s)'] = df_hr['hr_duration_(s)'].astype(float)

# get the maximum heart rate per day
df_hr['max_hr'] = df_hr.groupby(['participantId', 'startDate'])['value'].transform('max')

# get the minimum heart rate per day
df_hr['min_hr'] = df_hr.groupby(['participantId', 'startDate'])['value'].transform('min')

# get the average heart rate per day
df_hr['value'] = df_hr.groupby(['participantId', 'startDate'])['value'].transform('mean')



# Drop duplicates
df_hr.drop_duplicates(subset=['participantId', 'startDate'], inplace=True)


# Drop unnecessary columns
df_hr.drop(columns=['timestamp', 'startTime', 'endTime'], inplace=True)

# Rename value column
df_hr.rename(columns={'value': 'mean_hr/s'}, inplace=True)

print(df_hr)

                               participantId  mean_hr/s   startDate  \
0       06bc6ebb-a233-469f-8091-90256f656b1b   1.173082  2016-03-13   
382     2214f4fd-1ae0-4804-8663-b01c5f6d142c   1.260992  2016-03-02   
637     b2571643-4aec-492f-bc7f-6f23c7fe239a   1.263071  2018-10-03   
651     b1406c4e-e6ac-4297-a9e4-335ca5ef04de   1.203742  2019-01-24   
680     b1406c4e-e6ac-4297-a9e4-335ca5ef04de   1.903400  2019-01-23   
...                                      ...        ...         ...   
890164  a5fa37f1-de3c-41bd-9324-c31c5244e5c8   1.206158  2019-06-23   
890259  bb3b8938-47f4-42b5-a5f1-4b29fe5f5b99   0.953818  2019-06-25   
890275  bb3b8938-47f4-42b5-a5f1-4b29fe5f5b99   1.122333  2019-06-24   
890284  bb3b8938-47f4-42b5-a5f1-4b29fe5f5b99   1.072231  2019-06-28   
890300  bb3b8938-47f4-42b5-a5f1-4b29fe5f5b99   1.200000  2019-06-29   

        hr_duration_(s)  max_hr  min_hr  
0                 439.0   2.133   0.850  
382               247.0   1.783   1.050  
637                14

# Nap Tracker

Total duration

Average nap quality

In [12]:
df_nt = pd.read_csv(folder_path + 'nap-tracker.csv', usecols=['participantId', 'timestamp', 'NapDuration', 'NapQuality'], parse_dates=['timestamp'])

# calculate total nap duration per day for each participant

df_nt['timestamp'] = pd.to_datetime(df_nt['timestamp'], utc=True)
df_nt['startDate'] = df_nt['timestamp'].dt.date

# Remove rows where startDate, participantId and NapDuration are duplicated
df_nt.drop_duplicates(subset=['participantId', 'startDate', 'NapDuration'], inplace=True)

df_nt['totalNapTime_(s)'] = df_nt.groupby(['participantId', 'startDate'])['NapDuration'].transform('sum')

# Get the average nap quality per day for each participant

df_nt['avgNapQuality'] = df_nt.groupby(['participantId', 'startDate'])['NapQuality'].transform('mean')

# Drop unnecessary columns
df_nt.drop(columns=['timestamp', 'NapDuration', 'NapQuality'], inplace=True)

# If NaN is present in the avgNapQuality or totalNapTime column, fill with 0
df_nt['avgNapQuality'] = df_nt['avgNapQuality'].fillna(0)
df_nt['totalNapTime_(s)'] = df_nt['totalNapTime_(s)'].fillna(0)

print(df_nt.head())

                          participantId   startDate  totalNapTime_(s)  \
0  cb4c4df0-6685-4743-a3cf-d29c3d6941c9  2016-07-11            4500.0   
1  1f230684-4fa7-48ce-a7c2-6372238fe486  2016-09-07           24708.0   
2  02b96d81-c9f6-4a73-a075-75a64e54d005  2016-03-12            3300.0   
3  28e7a976-55e2-4ff1-8e95-7278f1ee9ec9  2016-09-07               0.0   
4  1dcf514b-8610-40f3-9a78-43ef80b5b7b7  2016-09-08            4093.0   

   avgNapQuality  
0            4.0  
1            4.0  
2            4.0  
3            3.0  
4            1.0  


# Step Count

In [13]:
df_sc = pd.read_csv(folder_path + 'step-count.csv', usecols=['participantId', 'startTime', 'endTime', 'value'], parse_dates=['startTime', 'endTime'])

# get total step count per day for each participant

df_sc['startTime'] = pd.to_datetime(df_sc['startTime'], utc=True)
df_sc['endTime'] = pd.to_datetime(df_sc['endTime'], utc=True)
df_sc['startDate'] = df_sc['startTime'].dt.date

df_sc['duration'] = df_sc['endTime'] - df_sc['startTime']
df_sc['duration'] = df_sc['duration'].dt.total_seconds()

df_sc['totalSteps'] = df_sc.groupby(['participantId', 'startDate'])['value'].transform('sum')
df_sc['totalSteps'] = df_sc['totalSteps'].astype(float)

# Get the total duration per day for each participant
df_sc['stepsTotalDuration'] = df_sc.groupby(['participantId', 'startDate'])['duration'].transform('sum')


# Get rid of duplicates

df_sc.drop_duplicates(subset=['participantId', 'startDate'], inplace=True)

# Drop unnecessary columns
df_sc.drop(columns=['value', 'duration', 'startTime', 'endTime'], inplace=True)

print(df_sc.head())

                            participantId   startDate  totalSteps  \
0    1f649060-680a-4c80-a551-be38ce46cb94  2016-03-22     10017.0   
21   1f649060-680a-4c80-a551-be38ce46cb94  2016-03-23     14752.0   
91   1f649060-680a-4c80-a551-be38ce46cb94  2016-03-24     15744.0   
116  1f649060-680a-4c80-a551-be38ce46cb94  2016-03-29     14477.0   
127  1f649060-680a-4c80-a551-be38ce46cb94  2016-03-30     12176.0   

     stepsTotalDuration  
0               10939.0  
21              13836.0  
91              13291.0  
116             15828.0  
127             12559.0  


# Distance Walking/Running

In [14]:
# Read in data

df_dwr = pd.read_csv(folder_path + 'distance-walking-running.csv', usecols=['participantId', 'timestamp', 'startTime', 'endTime', 'value'], parse_dates=['timestamp'])

# Convert timestamp to datetime and extract date

df_dwr['startTime'] = pd.to_datetime(df_dwr['startTime'], utc=True)
df_dwr['endTime'] = pd.to_datetime(df_dwr['endTime'], utc=True)
df_dwr['timestamp'] = pd.to_datetime(df_dwr['timestamp'], utc=True)

df_dwr['startDate'] = df_dwr['startTime'].dt.date

# Get the total distance walked/ran per day for each participant
df_dwr['totalDistance'] = df_dwr.groupby(['participantId', 'startDate'])['value'].transform('sum')

# Get the duration
df_dwr['duration'] = df_dwr['endTime'] - df_dwr['startTime']
df_dwr['duration'] = df_dwr['duration'].dt.total_seconds()

df_dwr['dwrTotalDuration'] = df_dwr.groupby(['participantId', 'startDate'])['duration'].transform('sum')

# Drop duplicates
df_dwr.drop_duplicates(subset=['participantId', 'startDate'], inplace=True)

# Drop unnecessary columns
df_dwr.drop(columns=['timestamp', 'startTime', 'endTime', 'duration', 'value'], inplace=True)


print(df_dwr.head())


                           participantId   startDate  totalDistance  duration  \
0   bfc93c66-90b4-4264-be63-b57c8b1a148f  2016-03-03       7140.476     360.0   
48  35fbae45-0f21-45ea-8ed7-8d3361a29b86  2016-06-16     103099.934      22.0   
49  41ec0944-d752-49c8-b6bd-b663abfd8f82  2016-12-06      34812.350      78.0   
50  4fee96d8-a09f-40f7-b8aa-132b062ae0a2  2016-05-23      14423.381      62.0   
55  3c6f9ed0-deaf-4996-8e26-e41787a525dd  2016-03-21         61.143      20.0   

    dwrTotalDuration  
0            12596.0  
48           47928.0  
49           99302.0  
50           16111.0  
55             140.0  


# Sleep Quality

For sleep quality, I will change the timestamp to just include the date, and ensure there are no duplicates

In [15]:
df_sq = pd.read_csv(folder_path + 'sleep-quality-checker.csv', usecols=['participantId', 'timestamp', 'value'], parse_dates=['timestamp'])

df_sq['timestamp'] = pd.to_datetime(df_sq['timestamp'], utc=True)

df_sq['startDate'] = df_sq['timestamp'].dt.date

# drop duplicates
df_sq.drop_duplicates(subset=['participantId', 'startDate'], inplace=True)

# drop unnecessary columns
df_sq.drop(columns=['timestamp'], inplace=True)

df_sq.rename(columns={'value': 'ssq_score'}, inplace=True)

print(df_sq.head())


                          participantId  ssq_score   startDate
0  b4ebf7dd-4e30-4f7b-8ee8-5493a19c8c9f          4  2016-09-27
1  c3993552-69cb-45e4-b18a-5e6eecefb825          4  2016-03-07
2  78f60bd3-34f3-489e-a352-f9df564641c3          4  2016-03-05
3  9da1a89a-2145-4cca-b356-7b58aa7be8b0          4  2016-09-27
4  4aad9dbe-dd9e-4832-a198-3bd563457124          4  2016-03-03


# Sampling Participants

The amount of rows in these dataframes are still too large, so I will randomly sample a portion of the participants.

I will first of all find the participants which have data in each dataframe, on the same days (except BMI).

Then I use a probability to keep a certain portion of the data.

In [16]:
import numpy as np

def random_sampling():
    # Create a list of DataFrames
    dfs_list = [df_bmi, df_energy, df_hr, df_nt, df_sc, df_dwr, df_sq]	


    # Create a list of all participants
    participants = set(dfs_list[0]['participantId'].unique())

    # Check that the common participants have a value for sleep quality score
    participants = participants.intersection(set(df_sq[~df_sq['ssq_score'].isnull()]['participantId'].unique()))

    #print(participants)

    # Probability of keeping a participant
    keep_probability = 0.7

    # Determine which participants to keep based on the probability
    participants_to_keep = np.random.choice(list(participants), size=int(len(participants) * keep_probability), replace=False)

    # Filter each original DataFrame to keep only the common participants
    filtered_dfs = [df[df['participantId'].isin(participants_to_keep)] for df in dfs_list]

    # Checking values
    for i, df in enumerate(filtered_dfs):

        #print(f'Columns of filtered_df[{i}]: {df.columns}')

        unique_participants = df['participantId'].unique()
        '''print(f"Unique participants in DataFrame {i}:")
        print(unique_participants)
        print(f"Number of unique participants: {len(unique_participants)}")
        print("\n" + "-"*30 + "\n")

        print(f"DataFrame {i}:")
        #print(df)
        print(f"Number of rows: {len(df)}")
        print("\n" + "-"*30 + "\n")'''




# Alternative to Randomly Sampling

For this alternative, rather than randomly sampling from the pool of participants that are common across all dataframes, I will first sort participants alphabetically in each dataframe, and add them to a list. 

Then, I will iterate through the first $i$ participants in that list, and merge the data present for those.

I will then get the next $i$ participants and perform a merge, and continute until there are $i$ or less participants - then I will merge those.

With each of these subsets, I will remove rows where there is missing data. 

Then, I will merge all of the complete subsets together, RAM permitting.
    If the data is still to large to perform a merge, I will randomly sample participants from the complete dataset

In [17]:
# merge function

def merge_dfs(dfs_list, participants):
    merged_df = None
    for df in dfs_list:
        # Filter dataframe for participants we currently work with
        curr_df = df[df['participantId'].isin(participants)]
        if merged_df is None:
            merged_df = curr_df
        else:
            merged_df = pd.merge(merged_df, curr_df, on=['participantId', 'startDate'], how='outer', suffixes=('', '_y'))
    return merged_df


In [18]:
# Another method of gathering a smaller dataset

def get_separate_tables(num_participants):
    
    # Create a list of DataFrames
    dfs_list = [df_bmi, df_hr, df_sc, df_dwr, df_sq]	

    # Create a list of all participants
    participants = set(dfs_list[0]['participantId'].unique())

    # Sort the list of participants alphabetically
    participants = sorted(list(participants))

    result_dfs = []

    while participants:
        # Get the next group of participants
        next_group = participants[:num_participants]

        # Merge the DataFrames for the next group of participants
        merge_df = merge_dfs(dfs_list, next_group)

        print(merge_df.head())

        # Add the merged DataFrame to the list of results
        result_dfs.append(merge_df)

        # Remove the participants that were just used
        participants = participants[num_participants:]

    return result_dfs


In [19]:
# Call get_separate_tables() and check the resulting dfs

subset_dfs = get_separate_tables(20)

for i, df in enumerate(subset_dfs):
    print(f"DataFrame {i}:")
    print(df.head())
    print(f"Number of rows: {len(df)}")
    print("\n" + "-"*30 + "\n")


                          participantId  bodyMass_kg  height_m   startDate  \
0  00a55fb5-da33-4e2e-ae61-28f589fcc174       51.256    1.5748  2016-05-12   
1  00d6d2ee-ccea-45c7-9772-b19fd9bef2bf       74.389    1.9050  2018-04-20   
2  00fd4039-9b5e-4bbb-8295-4983a3f58371       59.874    1.6510  2018-08-15   
3  0113e483-0fc8-4892-a4fc-0b2f3820dde3       68.039    1.7018  2019-03-06   
4  013d82d7-b3cc-4007-b00c-4d1d75bab9dd       76.204    1.6002  2018-02-21   

         bmi  active_energy  active_duration  basal_energy  basal_duration  \
0  20.667783            NaN              NaN           NaN             NaN   
1  20.498343            NaN              NaN           NaN             NaN   
2  21.965653         13.815           5193.0       227.847         14460.0   
3  23.493130            NaN              NaN           NaN             NaN   
4  29.759747            NaN              NaN           NaN             NaN   

   total_energy  ...  mean_hr/s  hr_duration_(s)  max_hr  min_

In [20]:
# For each unique participant, check if they have a value for weight, height, and bmi. If so, copy values to each row for that participant

for df in subset_dfs:

    # Loop through participants
    for participant in df['participantId'].unique():
        # Find the index of the row containing non NaN values for weight, height, and bmi
        non_nan_index = df[(df['participantId'] == participant) & (~df['bodyMass_kg'].isnull()) & (~df['height_m'].isnull()) & (~df['bmi'].isnull())].index

        # If there is a row with non NaN values, copy the values to all rows for that participant
        if len(non_nan_index) > 0:
            non_nan_index = non_nan_index[0]
            non_nan_row = df.loc[non_nan_index]
            df.loc[df['participantId'] == participant, 'bodyMass_kg'] = non_nan_row['bodyMass_kg']
            df.loc[df['participantId'] == participant, 'height_m'] = non_nan_row['height_m']
            df.loc[df['participantId'] == participant, 'bmi'] = non_nan_row['bmi']


# Save to CSV's

In [21]:
# Save the filtered DataFrames to new CSV files

save_path = 'C:\\Users\\aoife\Documents\\Project\\filtered_data\\'

'''for i, df in enumerate(filtered_dfs):
    if i == 0:
        df.to_csv(save_path + 'bmi.csv', index=False)
        print(f"DataFrame {i} has been saved to bmi.csv")
    elif i == 1:
        df.to_csv(save_path + 'energy.csv', index=False)
        print(f"DataFrame {i} has been saved to aeb.csv")
    elif i == 2:
        df.to_csv(save_path + 'hr.csv', index=False)
        print(f"DataFrame {i} has been saved to hr.csv")
    elif i == 3:
        df.to_csv(save_path + 'nt.csv', index=False)
        print(f"DataFrame {i} has been saved to nt.csv")
    elif i == 4:
        df.to_csv(save_path + 'sc.csv', index=False)
        print(f"DataFrame {i} has been saved to sc.csv")
    elif i == 5:
        df.to_csv(save_path + 'dwr.csv', index=False)
        print(f"DataFrame {i} has been saved to dwr.csv")
    elif i == 6:
        df.to_csv(save_path + 'sq.csv', index=False)
        print(f"DataFrame {i} has been saved to sq.csv")
 '''       
    

'for i, df in enumerate(filtered_dfs):\n    if i == 0:\n        df.to_csv(save_path + \'bmi.csv\', index=False)\n        print(f"DataFrame {i} has been saved to bmi.csv")\n    elif i == 1:\n        df.to_csv(save_path + \'energy.csv\', index=False)\n        print(f"DataFrame {i} has been saved to aeb.csv")\n    elif i == 2:\n        df.to_csv(save_path + \'hr.csv\', index=False)\n        print(f"DataFrame {i} has been saved to hr.csv")\n    elif i == 3:\n        df.to_csv(save_path + \'nt.csv\', index=False)\n        print(f"DataFrame {i} has been saved to nt.csv")\n    elif i == 4:\n        df.to_csv(save_path + \'sc.csv\', index=False)\n        print(f"DataFrame {i} has been saved to sc.csv")\n    elif i == 5:\n        df.to_csv(save_path + \'dwr.csv\', index=False)\n        print(f"DataFrame {i} has been saved to dwr.csv")\n    elif i == 6:\n        df.to_csv(save_path + \'sq.csv\', index=False)\n        print(f"DataFrame {i} has been saved to sq.csv")\n '

In [22]:
# Remove rows in each subset_df where there are NaN values

total_rows = 0

for i, df in enumerate(subset_dfs):
    subset_dfs[i] = subset_dfs[i].dropna()
    
    print(subset_dfs[i].head())
    # print the number of rows in the cleaned dataframe
    print(f"Number of rows in cleaned DataFrame {i}: {len(subset_dfs[i])}")

    #print the total rows
    total_rows += len(subset_dfs[i])

print(f"Total number of rows: {total_rows}")

                          participantId  bodyMass_kg  height_m   startDate  \
2  00fd4039-9b5e-4bbb-8295-4983a3f58371       59.874     1.651  2018-08-15   

         bmi  active_energy  active_duration  basal_energy  basal_duration  \
2  21.965653         13.815           5193.0       227.847         14460.0   

   total_energy  ...  mean_hr/s  hr_duration_(s)  max_hr  min_hr  totalSteps  \
2       241.662  ...   1.290195             41.0   1.517    1.15        61.0   

   stepsTotalDuration  totalDistance  duration  dwrTotalDuration  ssq_score  
2              1441.0          46.27      61.0            1441.0        4.0  

[1 rows x 21 columns]
Number of rows in cleaned DataFrame 0: 1
Empty DataFrame
Columns: [participantId, bodyMass_kg, height_m, startDate, bmi, active_energy, active_duration, basal_energy, basal_duration, total_energy, total_duration, mean_hr/s, hr_duration_(s), max_hr, min_hr, totalSteps, stepsTotalDuration, totalDistance, duration, dwrTotalDuration, ssq_score]
Ind

# Merge dataframes

In [23]:
# Concatenate the DataFrames in subset_dfs into a single DataFrame

concatenated_df = pd.concat(subset_dfs, ignore_index=True)

print(concatenated_df.head())

# print total number of rows in concatenated_df
print(f"Total number of rows in concatenated_df: {len(concatenated_df)}")


                          participantId  bodyMass_kg  height_m   startDate  \
0  00fd4039-9b5e-4bbb-8295-4983a3f58371       59.874    1.6510  2018-08-15   
1  0f279ffd-a1b5-473f-9cfc-01e5510ea5a4      111.584    1.8034  2018-06-19   
2  17485b1a-3fa5-496d-bc70-8fae55402149       93.894    1.8034  2017-12-19   
3  21b0fa2e-1270-4295-b3d2-01d23585a4fc       90.718    1.6764  2019-02-15   
4  26625914-859a-4a4d-8127-d77c2975aab5       92.986    1.7780  2018-02-28   

         bmi  active_energy  active_duration  basal_energy  basal_duration  \
0  21.965653         13.815           5193.0       227.847         14460.0   
1  34.309769         25.585           4781.0       140.331          5434.0   
2  28.870460         45.738           7749.0       662.984         32013.0   
3  32.280345         44.118           3785.0       904.345         41976.0   
4  29.413998         62.128           4707.0       800.720         29753.0   

   total_energy  ...  mean_hr/s  hr_duration_(s)  max_hr  min_

In [25]:
# Save the merged DataFrame to a new CSV file

concatenated_df.to_csv(save_path + 'concatenated_data.csv', index=False)